In [30]:
import tensorflow as tf 
import tensorflow_datasets as tfds

### Downloading and preprocessing the data

In [24]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 120 
NUM_EPOCHS = 20 

In [5]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [14]:
mnist_train_validation, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [16]:
def scale(img, label):
    img = tf.cast(img, tf.float64)
    img /= 255.
    return img, label

In [28]:
mnist_train_validation = mnist_train_validation.map(scale)
mnist_test = mnist_test.map(scale)

In [34]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_validation_test = mnist_info.splits['test'].num_examples
num_validation_test = tf.cast(num_validation_test, tf.int64)

In [ ]:
mnist_train_validation = mnist_train_validation.shuffle(BUFFER_SIZE)

In [32]:
mnist_train = mnist_train_validation.skip(num_validation_samples)
mnist_validation = mnist_train_validation.take(num_validation_samples)

In [33]:
mnist_train = mnist_train.batch(BATCH_SIZE)
mnist_validation = mnist_validation.batch(num_validation_samples)
mnist_test = mnist_train.batch(num_validation_test)

### Creating the model and training it

In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=50, kernel_size=5, activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                              
    tf.keras.layers.Conv2D(filters=50, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                              
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [45]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 24, 24, 50)        1300      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 50)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 50)        22550     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 50)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1250)              0         
                                                                 
 dense_1 (Dense)             (None, 10)               

In [51]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [59]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [54]:
early_stoping = tf.keras.callbacks.EarlyStopping(
    monitor= 'val_loss',
    mode='auto',
    min_delta=2,
    patience=0,
    restore_best_weights=True,
)

In [61]:
model.fit(
    mnist_train,
    epochs=NUM_EPOCHS,
    callbacks=[early_stoping],
    validation_data=mnist_validation,
    verbose=2
)

Epoch 1/20
450/450 - 12s - loss: 2.3014 - accuracy: 0.1125 - val_loss: 2.3013 - val_accuracy: 0.1113 - 12s/epoch - 26ms/step
Epoch 2/20
450/450 - 11s - loss: 2.3014 - accuracy: 0.1125 - val_loss: 2.3013 - val_accuracy: 0.1113 - 11s/epoch - 25ms/step
